## SpotifyAPI Client
This notebook contains a fully documented copy of the SpotifyAPI Class.

In [1]:
import requests
import base64
import datetime
import json
from urllib.parse import urlencode
from typing import Union

In [30]:
class SpotifyAPI(object):
    """A class to make authorised api requests to Spotify.
    
    Can perform authentication to obtain an access token for the spotify api, and then make requests of that api
    such as searching for artist, album, or track data.
    
      Typical usage example:
      
      spotify = SpotifyAPI(client_id, client_secret)
      results = spotify.search(query={"track": "money", "artist": "pink floyd"}, 
                              search_type="track", 
                              market_type="US")
    
    Attributes:
        access_token: a time-limited access token string provided by spotify for making api requests.
        access_token_expires: time at which current access_token expires.
        access_token_did_expire: expresses whether access_token has expired or not.
        client_id: spotify client id for the application.
        client_secret: spotify client secret for the application, relates to specific client id.
        token_url: url for obtaining spotify access token .
    """
    
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id: str, client_secret: str, *args, **kwargs) -> None:
        """Inits SpotifyAPI class and performs authentication.
        
        Args:
            client_id (str): spotify client id for the application.
            client_secret (str): spotify client secret for the application, relates to specific client id.
        """
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.perform_auth()
       
    
    def get_client_credentials(self) -> str:
        """Combines client_id and client_secret to create a single base64 encoded string.
        
        Returns:
            client_creds_b64.decode(): client-id and client_secret combined as a base64 encoded string.
        """
        
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    
    def get_token_headers(self) -> dict:
        """Creates headers for access token api request.
        
        Returns:
            A dict containing a formatted string for access token requests contining client credential information.
        """
        
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    
    def get_token_data(self) -> dict:
        """Creates token data as a dict.
                
        Returns:
            A dict containing access token request data, authorisation granted based on client credential information.
        """
        
        return {"grant_type" : "client_credentials"}
    
    
    def get_access_token(self) -> str:
        """Obtains and returns a valid access token.
        
        Returns:
            token(str): access token for making authenticated api requests to Spotify.
        """
        
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
     
        
    def perform_auth(self) -> bool:
        """Performs authentication for making api requests to Spotify.
        
        Returns:
            True once authentication has been successfully completed.
            
        Raises:
            Exception: Authentication failed - ensure client credentials are correct.
        """
        
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Authentication failed - ensure client credentials are correct.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    

    def get_resource_headers(self) -> dict:
        """Creates headers for resource api request.
        
        Returns:
            headers(dict): a dict containing access token for making  authorised api requests.
        """
        
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
    
    
    def get_resource(self, lookup_id: str, resource_type: str="track", version: str="v1") -> dict:
        """Makes an api request for resources from spotify.
        
        Retrieves JSON data related to resource request type and returns as a dict of 
        unaltered data.
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to a specific object 
              such as artist, track, album etc.
            resource_type(str): type of resource that relates to lookup_id such as 'track', 'artist', 
              'album' etc.
            version(str): a string relating to version of spotify api. At point of creation only v1 is 
              available.
        
        Returns:
            r.json() (dict): a dict containing unaltered data related to the requested resource. If 
              request is unsuccessful, returns an empty dict.  
        """
        
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    def get_artist(self, lookup_id: str) -> dict:
        """Gets artist data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific artist, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the artist. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id, resource_type="artists")
    
    
    def get_album(self, lookup_id: str) -> dict:
        """Gets album data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific album, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the album. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id, resource_type="albums")
    
    
    def get_track_features(self, lookup_id: str) -> dict:
        """Gets album data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific album, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the album. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id: str, resource_type="audio-features")

    
    def search(self, query: dict=None, search_type: str="artist", market_type: str="GB") -> dict:
        """Makes an api request for search data from spotify.
        
        Retrieves JSON data related to search request and returns as a dict of 
        unaltered data.
        
        Args:
            query(dict): the search query as a dict of parameters such as {'track': '<song_title>'}
            search_type(str): the type of search to be conducted such as for a 'track', 'artists', 
              'albums' etc.
            market_type(str): a string relating to market the searchable object is available in;
              "GB", "US" etc.

        Returns:
            r.json() (dict): a dict containing unaltered data related to the search request. If 
              request is unsuccessful, returns an empty dict.  
        """
        
        if query == None:
            raise Exception("A query is required")
        
        query_string=""
        for key,value in query.items():
            query_string += f"{key}:{value} "

        query_params = urlencode({"q" : query_string, "type" : search_type.lower(), "market" : market_type})
        lookup_url = f"https://api.spotify.com/v1/search?{query_params}"
        headers = self.get_resource_headers()
        
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if not r.status_code in range(200, 299):
            return {}
        return r.json()
   
    
    def get_tracks(self, query: dict=None) -> list:
        """Gets tracks from an api search request
        
        Retrieves JSON data related to search request and returns as a formatted list of dicts for
        easy lookup and manipulation. 
          
          Typical usage Example:
          
          results = SpotifyAPI.get_tracks({'track': 'money'})
          first_result = results[0]
          print(first_result)
          >>> {'track_id': _,
               'track_name: _,
               'artist': _,
               'track_url': _,
               'image_url': _}
        
        Args:
            query(dict): the search query as a dict of parameters such as {'track': '<song_title>'}

        Returns:
            tracks (list): a list of dicts containing formatted track data. See example above.
        """
        
        json_data = self.search(query, "track")
        try: 
            tracks = [{'track_id': i['id'], 
              'track_name': i['name'],
              'artist': i['artists'][0]['name'],
              'track_url': i['external_urls']['spotify'],
              'image_url': i['album']['images'][0]['url']
             } for i in json_data['tracks']['items']]
        except:
            printf("json_data dict not valid")
            return [] 
        return tracks
    
    
    def get_musical_data(self, track_id: str) -> dict:
        """Gets key and tempo info related to track
        
          Typical usage example:
          
          results = SpotifyAPI.get_musical_data("XXXXyyyyYYYYxxxxZZZZab")
          print(results)
          >>> {'track_id': XXXXyyyyYYYYxxxxZZZZab,
               'key': 'B Minor',
               'tempo': 120}
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific track, 
              usually obtained from SpotifyAPI.search().

        Returns:
            musical_data(dict): a dict containing formatted key & tempo information.
            
        Raises:
            Exception: No data found - check track_id
        """
        
        try:
            track_data = self.get_track_features(track_id)
            key = self.key_convert(track_data['key'], track_data['mode'])
            tempo = int(track_data['tempo'])
            musical_data = {"track_id": track_id,
                            "key": key,
                            "tempo": tempo}
            return musical_data
        except:
            raise Exception("No data found - check track_id")
        
    
    def key_convert(self, key: int, mode: int=None) -> str:
        """Converts key and mode value to musical key format (eg "B Minor")
        
        Args:
            key(int): a value that corresponds to one of 12 musical keys.
            mode(int): a value that indicates if major or minor key
        
        Returns:
            A formatted string that represents the musical key eg 'Gb', 'A Major', 'D Minor'
        """
        
        if key < 0 or key > 10:
            raise Exception("Invalid key value entered - bad Python joke")
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "E", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "A", 10: "Bb", 11: "B"}
        if mode == None:
            return keys[key]
        elif mode == 1:
            return f"{keys[key]} Major"
        else:
            return f"{keys[key]} Minor"      

### SEARCH
Let's try searching for something!

In [28]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = "d3ea7c74c5b64a8cb9f62029f3c41312"

In [29]:
# Create spotify Api instance
spotify = SpotifyAPI(client_id, client_secret)
# Search for tracks using multiple query arguements.
spotify.search(query={"track": "money"}, search_type="track", market_type="US")

track:money 
200


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Amoney+&type=track&market=US&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'},
       'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg',
       'id': '2YZyLoL8N0Wb9xBt1NhZWg',
       'name': 'Kendrick Lamar',
       'type': 'artist',
       'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6PBZN8cbwkqm1ERj2BGXJ1'},
     'href': 'https://api.spotify.com/v1/albums/6PBZN8cbwkqm1ERj2BGXJ1',
     'id': '6PBZN8cbwkqm1ERj2BGXJ1',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273d28d2ebdedb220e479743797',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02d28d2ebdedb220e479743797',
       'width': 300},
      {'height': 64,
       'url': 'h

In [18]:
# Get formatted musical data based on track id
spotify.get_musical_data("7Gx2q0ueNwvDp2BOZYGCMO")

{'track_id': '7Gx2q0ueNwvDp2BOZYGCMO', 'key': 'B Major', 'tempo': 124}

str